In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 130

In [60]:
!pip install --upgrade --q scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 26.4 MB/s eta 0:00:00


In [61]:
import numpy as np
import pandas as pd
from math import ceil
from scipy import linalg
from scipy.interpolate import interp1d
from scipy.linalg import lstsq
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA
from scipy.spatial import Delaunay
from sklearn.linear_model import LinearRegression, Ridge, ElasticNet
from sklearn.preprocessing import StandardScaler
import scipy.stats as stats
from sklearn.model_selection import train_test_split as tts, KFold
from sklearn.metrics import mean_squared_error as mse
from scipy.interpolate import interp1d, griddata, LinearNDInterpolator, NearestNDInterpolator
import statsmodels.api as sm
from math import ceil
from IPython.display import Image
from IPython.display import display
plt.style.use('seaborn-white')

# the following line(s) are necessary if you want to make SKlearn compliant functions
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted

In [162]:
scale = StandardScaler()

In [63]:
data = pd.read_csv('drive/MyDrive/Colab Notebooks/data441/cars.csv')

In [64]:
x = data.loc[:,'CYL':'WGT'].values
y = data['MPG'].values
xi = [[   4.,   79., 2074.],[   4.,   88., 2065.]]

# Adaptation of LOWESS with surpport for multi-dimensional features and different kernels
Based on the following papers:




>William S. Cleveland: "Robust locally weighted regression and smoothing
scatterplots", Journal of the American Statistical Association, December 1979,
volume 74, number 368, pp. 829-836.


>William S. Cleveland and Susan J. Devlin: "Locally weighted regression: An
approach to regression analysis by local fitting", Journal of the American
Statistical Association, September 1988, volume 83, number 403, pp. 596-610.
"""
##inspired by Alexandre Gramfort's univariate implementation 

original version of Gramfort's implementation of LOWESS: https://gist.github.com/agramfort/850437



In [66]:
# approach by Alex Gramfort

def lowess_ag(x, y, f=2. / 3., iter=3):
    """lowess(x, y, f=2./3., iter=3) -> yest
    Lowess smoother: Robust locally weighted regression.
    The lowess function fits a nonparametric regression curve to a scatterplot.
    The arrays x and y contain an equal number of elements; each pair
    (x[i], y[i]) defines a data point in the scatterplot. The function returns
    the estimated (smooth) values of y.
    The smoothing span is given by f. A larger value for f will result in a
    smoother curve. The number of robustifying iterations is given by iter. The
    function will run faster with a smaller number of iterations.
    """
    n = len(x) 
    r = int(ceil(f * n)) # decies how many point around the Xi is important
    # computes the smallest integer that is greater than or equal to f * n
    # multiply faraction with num obs, and truncating it and force it to int
    # instead of "tau" to control width of kernel, it controls the number of observations that's important(that's why it is foced int)
    
    h = [np.sort(np.abs(x - x[i]))[r] for i in range(n)]
    # sort the observatio based on the distance to Xi, only applys to univariant
    # iterates through X, get the distant of Xi to other points and and get the r th nearest neighbor
    # "r" number of observations nearest to Xi forms the neighborhood, the others are clips in next line

    w = np.clip(np.abs((x[:, None] - x[None, :]) / h), 0.0, 1.0) 
    # trick to creating square matrix of weights
    # cuts off values(values) other than the number of neighbor needed 
    #(will get 1s for the ones not needed, but tricubic kernels turns 1s to 0s)
    # w here contains the weights that will be used for lowess
    # he claims his approach has some form of built in robustification
    
    # to adapt it to higher dimention(here is univariant), 
    # will need to change .abs to ecludian diatance
    # also the trickery will not work

    w = (1 - w ** 3) ** 3  #-->tricubic kernel, can try different
         
    # introdues form of "rewighting" of weights
    # applying tricubic kernel to the weights

    # for multi dimensional, will need to change the above lines to something more like KNN
    # faiss: compute the k-nearest neighbors, or something from sklearn
    # https://github.com/facebookresearch/faiss
    # will also need to replace the kernels, Epanechnikov/gaussian(slower) for example
    # need to rewrite the lines h,w, which gets the neigborhood of k, which is a fraction of total number of data
    # and gets the weights

    # first goal would be given any multi-dimentional x,
    # if given a point x, how can i find the k-nearest neighbors?

    yest = np.zeros(n)
    delta = np.ones(n) # this creates adaptiveness
    for iteration in range(iter): # iter to apply rounds of readjusting of weights
        for i in range(n): # forces intercept no matter what
            weights = delta * w[:, i] # delta will just be 1
            b = np.array([np.sum(weights * y), np.sum(weights * y * x)])
            A = np.array([[np.sum(weights), np.sum(weights * x)], 
                          [np.sum(weights * x), np.sum(weights * x * x)]])
            beta = linalg.solve(A, b)  # this would be replaced as well

            yest[i] = beta[0] + beta[1] * x[i] # this makes the linear prediction based on lm regression

        residuals = y - yest # keeps the residuals
        s = np.median(np.abs(residuals)) # take median of residuals
        delta = np.clip(residuals / (6.0 * s), -1, 1) 
        # "standardize" the residuals, also cliping(discarding) residual outliers it based on magnitutde(-1,1)
        # "clips" the residuals that are too large or small
        # residuals that are bigger will get bigger weights, 
        # then those points will be given more importance(more influential)
        # re-adjustes the original weight
        
        delta = (1 - delta ** 2) ** 2

    return yest

# In order for this approach to be able to apply to multi-dimensional data, we need to adjust the way weights are calculated.

In the original adaptation code:<br><br>
**h = [np.sort(np.abs(x - x[i]))[r] for i in range(n)]**<br>
**w = np.clip(np.abs((x[:, None] - x[None, :]) / h), 0.0, 1.0)**
<br>
> This univariate implementation takes the adsolute differernce between the center point x and all other points(its neighbors) as distance, sorts it in increaseing order, and selects the top $f * n$, or $f$ fraction amount of data points that are closest to center point x to form a "neighborhood(or to "localize")

>Then, it applies the "tricubic" kernel transformation to form the weights.<br>
<br>
**w = (1 - w ** 3) ** 3**
<br>
$$ K(x):=\begin{cases}
(1-\|x\|^3)^3 \;\;\;if \;\;\; \|x\|<1 \\
0 \;\;\; \text{otherwise}
\end{cases}
$$

### **Other kernel functions such as 'gaussian' , 'epanechnikov' , 'quartic' also useable

## Changes for the adapted vectorized version:


**find_neighbors_tricubic(x, xi, n):**
>This function fines the ecludian distances between centered point $x_i$ and all other points,finds the **$n$ nearest neighbors** and applies the **Tricubic** kernel transformation to form the weights $w$

**weights_matrix_neighbors(x,x_new,kern,n):**
>This function goes through every point in matrix X, and forms the **weights matrix** by applying the given kernel to each point

## This version also predicts the data base on linear-interpolation


> When training, the algorithm makes prediction for everything point in X, then makes lines or hyperplanes(linera-interpolatioin) in order to make predictions on new points





In [77]:
pca = PCA(n_components=3)

## For X that has higher dimensions, drawing linear interpolations can be very slow, we can use PCA for dimensionality reduction before making interpolation

In [134]:
def lowess_regressor(X, y,x_new, kernel_type = 'tricubic',f=2. / 3., iter=3,intercept =True):
    #kernels and weight calculation functions
    # we need to compute all pairwise distances and figure out the k-nearest neightbors
    def tricubic_neighbors(x, xi, n):

        distances = np.sort(np.sqrt(np.sum((x - xi)**2, axis=1))) # axis = 1 means we are  calculating it column-wise
        max = distances[n]
        w = np.where(distances>max,0,70/81*(1-distances**3)**3)
        return w   
    def gaussian_neighbors(x, xi, n):

        distances = np.sort(np.sqrt(np.sum((x - xi)**2, axis=1)))
        max = distances[n]
        w = np.where(distances>max,0,1/(np.sqrt(2*np.pi))*np.exp(-1/2*distances**2))
        return w
    def epanechnikov_neighbors(x, xi, n):

        distances = np.sort(np.sqrt(np.sum((x - xi)**2, axis=1)))
        max = distances[n]
        w = np.where(distances>max,0,3/4*(1-distances**2))
        return w
    def quartic_neighbors(x, xi, n):

        distances = np.sort(np.sqrt(np.sum((x - xi)**2, axis=1)))
        max = distances[n]
        w = np.where(distances>max,0,15/16*(1-distances**2)**2)
        return w

    def weights_matrix_neighbors(x,x_new,kernel,n):
        if x_new.shape == 1:
            x_new=x_new.reshape(1,-1)
        length = len(x_new)
        return np.array([kernel(x,x_new[i],n) for i in range(length)])
    
    if kernel_type in ['tricubic','gaussian','epanechnikov','quartic']:
        if kernel_type == 'tricubic':
            kernel = tricubic_neighbors
        if kernel_type == 'gaussian':
            kernel = gaussian_neighbors  
        if kernel_type == 'epanechnikov':
            kernel = epanechnikov_neighbors    
        if kernel_type == 'quartic':
            kernel = quartic_neighbors
    else:
        raise ValueError("kernel type must be 'tricubic','gaussian','epanechnikov' or 'quartic' ")
    X, x_new, y =np.array(X), np.array(x_new), np.array(y)


    length = len(X) # get the length of x
    r = int(ceil(f * length)) # decies how many point around the Xi is important

    if len(y.shape)==1: # here we make column vectors
        y = y.reshape(-1,1)

    if len(X.shape)==1:
        X = X.reshape(-1,1)
        
    if intercept:
        X1 = np.column_stack([np.ones((len(X),1)),X])
    else:
        X1 = X

        # calculates the weights 
    w = weights_matrix_neighbors(X,X,kernel,r)
#this variant is predicting on the train
#the predict function uses linear interpolatioin to predict

    yest = np.zeros(length)
    delta = np.ones(length) # this creates adaptiveness

    for iteration in range(iter): # iter to apply rounds of readjusting of weights
        for i in range(length): # forces intercept no matter what
            W = np.diag(delta)* np.diag(w[:,i]) 
            b = np.transpose(X1).dot(W).dot(y)
            A = np.transpose(X1).dot(W).dot(X1)
            A = A + 0.001*np.eye(X1.shape[1]) # if we want L2 regularization
            #theta = linalg.solve(A, b) # A*theta = b
            beta, res, rnk, s = lstsq(A, b)
            yest[i] = np.dot(X1[i],beta)
        
        residuals = y- yest # keeps the residuals

        s = np.median(np.abs(residuals)) # take median of residuals

        delta = np.clip(residuals / (6.0 * s), -1, 1) 
            # "standardize" the residuals, also cliping(discarding) residual outliers it is is greater than 6 * median
            # residuals that are bigger will get bigger weights, 
            # then those points will be given more importance(more influential)
            # re-adjustes the original weight
            # how robustification is applied
            
        delta = (1 - delta ** 2) ** 2
        
        
    if X.shape[1]==1:
        f = interp1d(x.flatten(),yest,fill_value='extrapolate')
        output = f(x_new)
    else:
        output = np.zeros(len(x_new))
        for i in range(len(x_new)):
            ind = np.argsort(np.sqrt(np.sum((X-x_new[i])**2,axis=1)))[:r]
            # the trick below is to have the Delaunay triangulation work --> getting first 3 PCA principle components
            #(often without this delaunay would be infinite loop), bypass deficiency in delaunay function
            pca = PCA(n_components=3)
            X_pca = pca.fit_transform(X[ind])
            tri = Delaunay(X_pca,qhull_options='QJ')
            f = LinearNDInterpolator(tri,y[ind]) 
            output[i] = f(pca.transform(np.array(x_new[i]).reshape(1,-1))) # the output may have NaN's where the data points from xnew are outside the convex hull of X 
    if sum(np.isnan(output))>0:
        g = NearestNDInterpolator(X,y.ravel()) 
        # output[np.isnan(output)] = g(X[np.isnan(output)])
        output[np.isnan(output)] = g(x_new[np.isnan(output)])

    return output

## Create a class that could be used in sk-learn's sytax

In [143]:
class Lowess_AG_adapted:
    def __init__(self, f = 1/10, kernel_type = 'tricubic',iter = 3,intercept=True):
        self.f = f
        self.iter = iter
        self.intercept = intercept
        self.kernel_type = kernel_type
    
    def fit(self, x, y):
        f = self.f
        iter = self.iter
        kernel_type = self.kernel_type
        self.xtrain_ = x
        self.yhat_ = y

    def predict(self, x_new):
        check_is_fitted(self)
        x = self.xtrain_
        y = self.yhat_
        f = self.f
        iter = self.iter
        intercept = self.intercept
        kernel_type = self.kernel_type
        return lowess_regressor(x, y, x_new, kernel_type,f, iter, intercept)

    def get_params(self, deep=True):
    # suppose this estimator has parameters "f", "iter" and "intercept"
        return {"f": self.f, "iter": self.iter,"intercept":self.intercept, "kernel_type":self.kernel_type}

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

## Simple testing withAutomobile data from UCI
https://archive.ics.uci.edu/ml/datasets/automobile

In [148]:
data = pd.read_csv('drive/MyDrive/Colab Notebooks/data441/cars.csv')

x = data.loc[:,'CYL':'WGT'].values
y = data['MPG'].values

x_scaled = scale.fit_transform(x)

In [92]:
# k-fold validation
def validation_function(x,y,model):
  kf = KFold(n_splits=10,shuffle=True,random_state=123)
  mse_test_lowess = []
  for idxtrain, idxtest in kf.split(x):
    xtrain = scale.fit_transform(x[idxtrain])
    xtest = scale.transform(x[idxtest])
    ytrain = y[idxtrain]
    ytest = y[idxtest]
    # for our 1-dimensional input data we do not need scaling
    model.fit(xtrain,ytrain)
    mse_test_lowess.append(mse(ytest,model.predict(xtest)))
  return np.mean(mse_test_lowess)

In [163]:
validation_function(x,y,Lowess_AG_adapted())

26.614787944873125

# Using Grid search CV from sk-learn

In [160]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split as tts, KFold, GridSearchCV
lwr_pipe = Pipeline([('zscores', StandardScaler()),
                     ('lwr', Lowess_AG_adapted())])
params = [{'lwr__f': [1/i for i in range(3,15)],
         'lwr__iter': [1,2,3,4]}]


gs_lowess = GridSearchCV(lwr_pipe,
                      param_grid=params,
                      scoring='neg_mean_squared_error',
                      cv=5)
gs_lowess.fit(x, y)
gs_lowess.best_params_

{'lwr__f': 0.3333333333333333, 'lwr__iter': 1}

In [161]:
gs_lowess.best_score_

-29.934237859286906

[[4.0, 79.0, 2074.0], [4.0, 88.0, 2065.0]]

(2, 392)

# for each points from the test set would have 1 weight vector based on distances and kernel transformations.

# need to solve the weighted linear systems

[[4.0, 79.0, 2074.0], [4.0, 88.0, 2065.0]]

(2, 392)